In [ ]:
'''
conceptual notes
    goal is to create script that takes in a message and encrypts it using the rlce encryption method
    
    for the sake of simplicity, this code works with a binary matrix defined by GF(2)

    definitions
        m - message vector of length k
        n - codeword length
        t - number of errors that linear code can correct
        S - non singular matrix, k X k
        P - permutation matrix, n X n
        r - number of random columns added
        public key - how messages are encrypted; G = S *Gs * P
        Gs - private key, generator matrix k X n
        codeword - message embedded into generator matrix
        ciphertext - codeword + some errors to further hide it; decrypted with the private key

    steps
        1. create random generator matrix Gs to create linear code
        2. add r number of columns to Gs to create G1, to hide Gs
        3. create matrix A to scramble the COLUMNS of G1 in blocks (chunks)
        4. generate random invertible matrix S to scramble the ROWS of G1
        5. generate matrix P to shuffle columns of G1
        6. generate public key
        7. pass back private key

'''

import numpy as np
import galois

GF2 = galois.GF(2) # defining GF(2) finite field

# function to make random matrix with specified num of rows/columns, within GF(2)
def randomMatrix(rows, cols):
    return np.random.randint(0,2, size=(rows, cols))

# function to perform matrix multiplacation and keep answers within GF(2)
def formCodeword(m, G):
    return np.mod(np.dot(m, G), 2)

# function to create 
def randomInvertibleMatrix(n):
    while True:
        M = randomMatrix(n, n)
        try:
            _ = GF2(M).inverse() 
            return M
        except:
            continue

# function to make inverse of GF(2) matrix 
def inverseMatrix(matrix):
    m = GF2(matrix) # converting numpy matrix to datatype applicable to galois library inverse operation
    m_inv = m.inverse()
    return np.array(m_inv, dtype=int)

# function to decrypt matrix
def decodeMatrix(ciphertext, G):
    G_inv = GF2.inverse(G)
    return np.mod(np.dot(ciphertext, G_inv), 2)


'''
rlce encryption process  
'''

def keyGenerator(k, n, t, r):
    G = randomMatrix(k, n)
    S = randomInvertibleMatrix(k)
    P = np.eye(n*(r+1), dtype=int)[np.random.permutation(n*(r+1))]

    G1_columns = []
    for col in range(n):
        G1_columns.append(G[:, col:col+1])
        for _ in range(r):
            G1_columns.append(randomMatrix(k, 1))
    G1 = np.hstack(G1_columns)

    def blockDiagonalRandomMatrices(n_blocks, block_size):
        blocks = [randomInvertibleMatrix(block_size) for _ in range(n_blocks)]
        return GF2(np.block([[blocks[i] if i == j else np.zeros((block_size, block_size), dtype=int)
                              for j in range(n_blocks)]
                              for i in range(n_blocks)]))
    
    A = blockDiagonalRandomMatrices(n, r+1)

    G_pub = np.mod(S @ G1 @ A @ P, 2)
    private_key = {"S": S, "P": P, "A": A, "G1": G1, "k": k, "n": n, "r": r}
    return G_pub, private_key

def encrypt(G_pub, m, t):
    codeword = formCodeword(m, G_pub)
    e = np.zeros(codeword.shape, dtype=int) # creating vector to hold errors the same length as the codeword
    e[np.random.choice(len(codeword), size=t, replace=False)] = 1 # randomly switching a number of t positions in the vector from 0 to 1
    return np.mod(codeword + e, 2)

def decrypt(priv_key, ciphertext):
    S, P, A, G1, k = priv_key["S"], priv_key["P"], priv_key["A"], priv_key["G1"], priv_key["k"]
    P_inv = inverseMatrix(P)
    ctextPerm = np.mod(ciphertext @ P_inv @ inverseMatrix(A), 2) # undoing column perm/A scrambling
    for m_candidate in range(2**k): # for loop to search for message that encodes to ctextPerm by checking against private generator
        m_vec = np.array([int(b) for b in format(m_candidate, f"0{k}b")])
        if np.array_equal(formCodeword(m_vec, G1), ctextPerm):
            return np.mod(m_vec @ inverseMatrix(S), 2) # if message exists, undo row scramble
    raise ValueError("Decoding failed")


